# Exit pool description

As explained in [Balancer Docs](https://docs.balancer.fi/reference/joins-and-exits/pool-exits.html), the exit pool has 6 arguments:
- poolId - ID of the pool you're interacting with
- sender - Address sending BPT
- recipient - Address receiving tokens (usually the same as sender)
- request - ExitPoolRequest tuple made up of the following:
    - assets - List of your tokens, ordered. When providing your assets, you must ensure that the tokens are sorted numerically by token address. It's also important to note that the values in minAmountsOut correspond to the same index value in assets, so these arrays must be made in parallel after sorting.
    - minAmountsOut - Minimum token receive amounts. In the exitPool call, you have to provide minAmountsOut, the lower limits for the tokens to receive. In short, what are the minimum amounts you would find acceptable, given the amount of BPT you are providing?
    - userData - Custom bytes field (see below)
    - toInternalBalance - True if you receiving tokens as internal token balances. False if receiving as ERC20.

There is 4 kinds of exit pools:
- EXACT_BPT_IN_FOR_ONE_TOKEN_OUT: User sends a precise quantity of BPT, and receives an estimated but unknown (computed at run time) quantity of a single token.
- EXACT_BPT_IN_FOR_TOKENS_OUT: User sends a precise quantity of BPT, and receives an estimated but unknown (computed at run time) quantities of all tokens.
- BPT_IN_FOR_EXACT_TOKENS_OUT: User sends an estimated but unknown (computed at run time) quantity of BPT, and receives precise quantities of specified tokens.
- MANAGEMENT_FEE_TOKENS_OUT: This is used as an internal function only, so it is out of the scope.

Now, let's try to understand the parameters for each kind of transfer. As example, let's say that the user wants to leave the [80BAL-20WETH Pool](https://app.balancer.fi/#/ethereum/pool/0x5c6ee304399dbdb9c8ef030ab642b10820db8f56000200000000000000000014). So, the fixed values will be:
- pool_id: 0x5c6ee304399dbdb9c8ef030ab642b10820db8f56000200000000000000000014
- sender: $user_address
- recipient: $user_adress
- request.toInternalBalance: True #(the user can set it but this will be the default).
- request.minAmountOut: 0,0 #(the user can set it but this will be the default).

In [1]:
weth_address = "0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2"
bal_address = "0xba100000625a3754423978a60c9317c58a424e3d"
request = {
    "assets":sorted([weth_address, bal_address]),
}

request

{'assets': ['0xba100000625a3754423978a60c9317c58a424e3d',
  '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2']}

This means that the bal is the first asset and the weth is the second asset

## EXACT_BPT_IN_FOR_ONE_TOKEN_OUT

- userData ABI: ['uint256', 'uint256', 'uint256']
- userData: [EXACT_BPT_IN_FOR_ONE_TOKEN_OUT, bptAmountIn, exitTokenIndex]


In [11]:
# from balpy

from enum import IntEnum

class WeightedPoolJoinKind(IntEnum):
  INIT = 0
  EXACT_TOKENS_IN_FOR_BPT_OUT = 1
  TOKEN_IN_FOR_EXACT_BPT_OUT = 2
  ALL_TOKENS_IN_FOR_EXACT_BPT_OUT = 3

class WeightedPoolExitKind(IntEnum):
  EXACT_BPT_IN_FOR_ONE_TOKEN_OUT = 0
  EXACT_BPT_IN_FOR_TOKENS_OUT = 1
  BPT_IN_FOR_EXACT_TOKENS_OUT = 2
  MANAGEMENT_FEE_TOKENS_OUT = 3


In [17]:
import eth_abi

transaction_type = "EXACT_BPT_IN_FOR_ONE_TOKEN_OUT"
bptAmountIn = 1
exitTokenIndex = 1 # WETH

user_data = eth_abi.encode(['uint256', 'uint256', 'uint256'],
                           [WeightedPoolExitKind[transaction_type], bptAmountIn, exitTokenIndex]);
user_data


b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01'

## EXACT_BPT_IN_FOR_TOKENS_OUT

- userData ABI: ['uint256', 'uint256']
- userData: [EXACT_BPT_IN_FOR_TOKENS_OUT, bptAmountIn]

In [28]:
transaction_type = "EXACT_BPT_IN_FOR_TOKENS_OUT"
bptAmountIn = 1

user_data = eth_abi.encode(['uint256', 'uint256'],
                           [WeightedPoolExitKind[transaction_type], bptAmountIn]);
user_data

b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01'

## BPT_IN_FOR_EXACT_TOKENS_OUT

- userData ABI: ['uint256', 'uint256[]', 'uint256']
- userData: [BPT_IN_FOR_EXACT_TOKENS_OUT, amountsOut, maxBPTAmountIn]


In [27]:
transaction_type = "BPT_IN_FOR_EXACT_TOKENS_OUT"
amountsOut = [1432, 2423]
maxBPTAmountIn = 42341 # the user should set or we should provide a default (max)?

user_data = eth_abi.encode(['uint256', 'uint256[]','uint256'],
                           [WeightedPoolExitKind[transaction_type], amountsOut, maxBPTAmountIn])
user_data


b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00`\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xa5e\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x05\x98\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\tw'

# Function example

In [75]:
import balpy
import os
from web3 import Web3
from typing import List

from abc import ABC, abstractmethod, abstractproperty
import enum

class WeightedPoolExitKindAbstracted(ABC):
  @staticmethod
  def getDefaultMinAmountsOut(token_list):
    return [0] * len(token_list)

  @abstractproperty
  def id():
    pass
  
  @abstractmethod
  def getUserData():
    pass

  @abstractmethod
  def sortTokenList():
    pass

  @abstractmethod
  def getExitPoolRequestTuple():
    pass


class ExactBptInForOneTokenOut(WeightedPoolExitKindAbstracted):
  id = 0

  def sortTokenList(self, w3, token_list, token_out, min_amounts_out):
    token_list_address = [w3.toChecksumAddress(token) for token in token_list]
    min_amounts_out_sorted = [minAmountOut for _,minAmountOut in sorted(zip(token_list_address, min_amounts_out))]
    token_out_address = w3.toChecksumAddress(token_list_address[token_out])
    token_list_address_sorted = sorted(token_list_address)
    token_out_sorted = token_list_address_sorted.index(token_out_address)
    return token_list_address_sorted, token_out_sorted, min_amounts_out_sorted

  def getUserData(self, bpt_amount, token_out):
    return eth_abi.encode(['uint256', 'uint256', 'uint256'], [id, bpt_amount, token_out])  

  def getExitPoolRequestTuple(self, w3, token_list, bpt_amount, token_out, to_internal_balance: bool = True, min_amounts_out: List[int]= None):
    if min_amounts_out is None:
      min_amounts_out = self.getDefaultMinAmountsOut(token_list)
    token_list, token_out, min_amounts_out = self.sortTokenList(w3, token_list, token_out, min_amounts_out)
    user_data = self.getUserData(bpt_amount, token_out)
    return (token_list, min_amounts_out, user_data, to_internal_balance)


class WeightedPoolExitKind(enum.Enum):
  EXACT_BPT_IN_FOR_ONE_TOKEN_OUT = ExactBptInForOneTokenOut()

class BalpyBleu():
	def __init__(self, **balpy_kwargs):
		self.web3 = Web3(Web3.HTTPProvider(f"https://eth.llamarpc.com/rpc/{os.getenv('LLAMA_PROJECT_ID')}"))
		self.balpy = balpy.Balpy(self.web3, **balpy_kwargs)


	def balDoExitPool(self, poolId, address, exitPoolRequestTuple, gasFactor=1.05, gasPriceSpeed="average", nonceOverride=-1, gasEstimateOverride=-1, gasPriceGweiOverride=-1):
		vault = self.balLoadContract("Vault");
		exitPoolFunction = vault.functions.exitPool(poolId, address, address, exitPoolRequestTuple);
		tx = self.buildTx(exitPoolFunction, gasFactor, gasPriceSpeed, nonceOverride, gasEstimateOverride, gasPriceGweiOverride);
		print("Transaction Generated!");
		txHash = self.sendTx(tx);
		return(txHash);

	def balExitPool(
		self, 
		poolId: str,
		address: str,
		bptAmount: int,
		tokenList: List[str],
		exitType:str, 
		tokenOut:int = None,
		amountsOut:List[int] = [],
		query:bool = False,
		**balDoExitPoolKwargs):
		"""
		poolId: Id of the pool to exit
		address: Address of the user to burn BPTs from and receive tokens to
		bpt_amount: Amount of BPTs to burn (if exit_type == "EXACT_BPT_IN_FOR_ONE_TOKEN_OUT" or "EXACT_BPT_IN_FOR_TOKENS_OUT")
		  		    or max amount of BPTs to burn (if exit_type == "BPT_IN_FOR_EXACT_TOKENS_OUT")
		token_list: List of token addresses of the pool
		exit_type: "EXACT_BPT_IN_FOR_ONE_TOKEN_OUT", "EXACT_BPT_IN_FOR_TOKENS_OUT", or "BPT_IN_FOR_EXACT_TOKENS_OUT"
		token_out: Index of the token (related to token_list order) to receive (if exit_type == "EXACT_BPT_IN_FOR_ONE_TOKEN_OUT")
		amounts_out: List of amounts of tokens (same order of token_list) to receive (if exit_type == "BPT_IN_FOR_EXACT_TOKENS_OUT")
		query: If True, returns the transaction object without sending the transaction
		"""

		# self.checkExitPoolTypeAndArgumentsMatch(bpt_amount, token_list, exit_type, token_out, amounts_out)
		exitPoolRequestTuple = WeightedPoolExitKind[exitType].value.
		return
	


SyntaxError: invalid syntax (3771093709.py, line 43)

In [73]:
from abc import ABC, abstractmethod, abstractproperty
import enum

class WeightedPoolExitKindAbstracted(ABC):
  @staticmethod
  def getDefaultMinAmountsOut(token_list):
    return [0] * len(token_list)

  @abstractproperty
  def id():
    pass
  
  @abstractmethod
  def getUserData():
    pass

  @abstractmethod
  def sortTokenList():
    pass

  @abstractmethod
  def getExitPoolRequestTuple():
    pass


class ExactBptInForOneTokenOut(WeightedPoolExitKindAbstracted):
  id = 0

  def sortTokenList(self, w3, token_list, token_out, min_amounts_out):
    token_list_address = [w3.toChecksumAddress(token) for token in token_list]
    min_amounts_out_sorted = [minAmountOut for _,minAmountOut in sorted(zip(token_list_address, min_amounts_out))]
    token_out_address = w3.toChecksumAddress(token_list_address[token_out])
    token_list_address_sorted = sorted(token_list_address)
    token_out_sorted = token_list_address_sorted.index(token_out_address)
    return token_list_address_sorted, token_out_sorted, min_amounts_out_sorted

  def getUserData(self, bpt_amount, token_out):
    return eth_abi.encode(['uint256', 'uint256', 'uint256'], [id, bpt_amount, token_out])  

  def getExitPoolRequestTuple(self, w3, token_list, bpt_amount, token_out, to_internal_balance: bool = True, min_amounts_out: List[int]= None):
    if min_amounts_out is None:
      min_amounts_out = self.getDefaultMinAmountsOut(token_list)
    token_list, token_out, min_amounts_out = self.sortTokenList(w3, token_list, token_out, min_amounts_out)
    user_data = self.getUserData(bpt_amount, token_out)
    return (token_list, min_amounts_out, user_data, to_internal_balance)


class WeightedPoolExitKind(enum.Enum):
  EXACT_BPT_IN_FOR_ONE_TOKEN_OUT = ExactBptInForOneTokenOut()

In [59]:
tester = BalpyBleu()
tester.balExitPool(poolId="", address="", bpt_amount=2, token_list=["", ""], exit_type="EXACT_BPT_IN_FOR_ONE_TOKEN_OUT", token_out=1)

In [48]:
web3.checkSumAddress("")

AttributeError: module 'web3' has no attribute 'checkSumAddress'

In [74]:
WeightedPoolExitKind.EXACT_BPT_IN_FOR_ONE_TOKEN_OUT.value.getExitPoolRequestTuple('', '', '', '')

AttributeError: 'str' object has no attribute 'toChecksumAddress'